## Machine Learning With Flowers

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import EfficientNetB3
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adamax

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [2]:
data = []

for dir, x, files in os.walk("flower_photos"):
    label = dir.split('/')[-1]
    # print(dir)
    # print(files)
    for file in files:
        path = os.path.join(dir,file)
        data.append([path,label])

#print(data)
df = pd.DataFrame(data, columns = ["file_path", "label"])
print(df)

                                             file_path          label
0                             flower_photos/.gitignore  flower_photos
1       flower_photos/roses/16209331331_343c899d38.jpg          roses
2        flower_photos/roses/5777669976_a205f61e5b.jpg          roses
3      flower_photos/roses/4860145119_b1c3cbaa4e_n.jpg          roses
4       flower_photos/roses/15011625580_7974c44bce.jpg          roses
...                                                ...            ...
3666     flower_photos/tulips/134143359_71fa8dd9a4.jpg         tulips
3667    flower_photos/tulips/3637371174_a8dfcc1b35.jpg         tulips
3668  flower_photos/tulips/6948239566_0ac0a124ee_n.jpg         tulips
3669    flower_photos/tulips/2834890466_1cf220fba1.jpg         tulips
3670   flower_photos/tulips/13953090784_0c7d7a904e.jpg         tulips

[3671 rows x 2 columns]


In [3]:
datagen = ImageDataGenerator(
    validation_split = 0.2
)

In [4]:
train_gen = datagen.flow_from_dataframe(
    dataframe = df,
    x_col = 'file_path',
    y_col = 'label',
    target_size=(224,224),
    color_mode = 'rgb',
    batch_size = 32,
    class_mode = 'categorical',
    shuffle = True,
    subset = 'training'
)

Found 2936 validated image filenames belonging to 5 classes.


/Users/Cohort23/Library/Python/3.10/lib/python/site-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 1 invalid image filename(s) in x_col="file_path". These filename(s) will be ignored.
  warnings.warn(


In [5]:
valid_gen = datagen.flow_from_dataframe(
    dataframe = df,
    x_col = 'file_path',
    y_col = 'label',
    target_size=(224,224),
    color_mode = 'rgb',
    batch_size = 32,
    class_mode = 'categorical',
    shuffle = True,
    subset = 'validation'
)

Found 734 validated image filenames belonging to 5 classes.


/Users/Cohort23/Library/Python/3.10/lib/python/site-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 1 invalid image filename(s) in x_col="file_path". These filename(s) will be ignored.
  warnings.warn(


In [6]:
base_model = EfficientNetB3(
    include_top= False, 
    weights='imagenet',
    input_shape=(224, 224, 3), 
    pooling='max'
)

model = tf.keras.models.Sequential([
    base_model,
    layers.BatchNormalization(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(rate=.45, seed=123),
    layers.Dense(5, activation='softmax')
])


model.compile(
    Adamax(learning_rate= .0001), 
    loss = 'categorical_crossentropy',
    metrics=[
        'acc', 
        tf.keras.metrics.Precision(), 
        tf.keras.metrics.Recall(), 
        tf.keras.metrics.AUC()
    ]
)

In [7]:
early_stopping = EarlyStopping(
    patience =10,
    min_delta = 0,
    monitor = 'val_loss',
    verbose = 0,
    restore_best_weights= True,
    baseline=None
)

In [30]:
model.fit(
    train_gen,
    validation_data=valid_gen,
    epochs = 5,
    validation_steps=None,
    shuffle=False,
    callbacks=early_stopping,
    verbose=1
)

Epoch 1/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - acc: 0.5901 - auc: 0.8511 - loss: 1.1304 - precision: 0.6759 - recall: 0.5085

/Users/Cohort23/Library/Python/3.10/lib/python/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


92/92 ━━━━━━━━━━━━━━━━━━━━ 294s 3s/step - acc: 0.5904 - auc: 0.8513 - loss: 1.1298 - precision: 0.6760 - recall: 0.5089 - val_acc: 0.1294 - val_auc: 0.5246 - val_loss: 2.6597 - val_precision: 0.1225 - val_recall: 0.0926
Epoch 2/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 313s 3s/step - acc: 0.7670 - auc: 0.9452 - loss: 0.6599 - precision: 0.8208 - recall: 0.7070 - val_acc: 0.1172 - val_auc: 0.5167 - val_loss: 3.2108 - val_precision: 0.1309 - val_recall: 0.1104
Epoch 3/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 295s 3s/step - acc: 0.8371 - auc: 0.9742 - loss: 0.4456 - precision: 0.8836 - recall: 0.8060 - val_acc: 0.1158 - val_auc: 0.4987 - val_loss: 3.7899 - val_precision: 0.1221 - val_recall: 0.1090
Epoch 4/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 289s 3s/step - acc: 0.8825 - auc: 0.9835 - loss: 0.3511 - precision: 0.9123 - recall: 0.8497 - val_acc: 0.1158 - val_auc: 0.4799 - val_loss: 4.1501 - val_precision: 0.1205 - val_recall: 0.1090
Epoch 5/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 289s 3s/step - acc: 0.9039 - auc: 0.9899 - loss: 0.27

In [31]:
model.save('flowers.keras')

In [9]:
import cv2
from tensorflow.keras.saving import load_model
import numpy as np

model = load_model("flowers.keras")

input_image = cv2.imread('flower_photos/daisy/100080576_f52e8ee070_n.jpg')

input_image_resize = cv2.resize(input_image, (224,224))
input_image_scaled = input_image_resize/225
image_reshaped = np.reshape(input_image_scaled, (1,224,224,3))

print(train_gen.class_indices.items())

input_prediction = model.predict(image_reshaped)
print("Your flower is a:", np.argmax(input_prediction))

dict_items([('daisy', 0), ('dandelion', 1), ('roses', 2), ('sunflowers', 3), ('tulips', 4)])
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Your flower is a: 0
